In [1]:
from bs4 import BeautifulSoup 
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

from sites import NASA_Mars_News_Site, JPL_Featured_Space_Image, Mars_Facts, USGS_Astrogeology_site

ModuleNotFoundError: No module named 'webdriver_manager'

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

NameError: name 'ChromeDriverManager' is not defined

In [3]:
browser.visit(NASA_Mars_News_Site)

NameError: name 'browser' is not defined

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
gallery = soup.find(class_='grid_gallery')
news_title = gallery.find(class_="content_title").text
news_p = gallery.find(class_='article_teaser_body').text
news_p

NameError: name 'browser' is not defined

In [5]:
browser.visit(JPL_Featured_Space_Image)
JPL_Featured_Space_Image

NameError: name 'browser' is not defined

In [6]:
# Handle search
browser.find_by_tag('input').first.fill('Featured Mars')

NameError: name 'browser' is not defined

In [7]:
# Handle Sortby
browser.find_by_id('searchHelpers_sortBy').first.select('latestDate')

NameError: name 'browser' is not defined

In [8]:
results = browser.find_by_id('SearchListingPageResults')

NameError: name 'browser' is not defined

In [9]:
results.links.find_by_partial_href('images').click()
browser.find_by_text('Download JPG ').click

NameError: name 'results' is not defined

In [10]:
featured_image_url = browser.url

NameError: name 'browser' is not defined

In [11]:
browser.visit(JPL_Featured_Space_Image)
results = browser.find_by_css('.SearchResultCard').first
results.html

NameError: name 'browser' is not defined